# Merge all the monthly files into a single set of references and compress

In [3]:
from kerchunk.combine import MultiZarrToZarr
import glob
import json
import fsspec

In [5]:
jsons = glob.glob('/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/*NCCF*.json')
jsons

['/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/202104NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json',
 '/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/202105NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json',
 '/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/202106NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json',
 '/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/202107NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json',
 '/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/202108NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json',
 '/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/202109NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json',
 '/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/202110NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json',
 '/home/jovyan/kerchunk_indexes/noaa-himawari8/A

In [6]:
ref_dicts=[]
for j in jsons:
    with open(j) as f:
        ref_dicts.append(json.load(f))

In [7]:
storage_options=dict(target_options={},
                    target_protocol='local',
                    remote_options=dict(anon=True),
                    remote_protocol='s3')

mzz = MultiZarrToZarr(ref_dicts,
                    remote_protocol="s3",
                    remote_options=storage_options,
                    concat_dims=["time"], coo_map={"time": "data:time"},
                    preprocess=None)

In [8]:
dict_agg = mzz.translate()

In [10]:
with fsspec.open('/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/combined.zstd',compression='zstd',mode='wb') as f:
    f.write(json.dumps(dict_agg).encode())